# Sentiment Analysis of Amazon Customer Reviews

This project uses the customer review data from Amazon.com Kindle store to perform a supervised binary (positive or negative) sentiment classification analysis. We use various data pre-processing techniques and three machine learning models, namely, Naive Bayes classification model, the Logistic regression model, and the linear support vector classification model. The result provides 87% prediction accuracy.

In [3]:
import pyspark
spark.conf.set('spark.sql.shuffle.partitions', '8')

### Load dataset
The data comes from the website "Amazon product data" (http://jmcauley.ucsd.edu/data/amazon/) managed by Dr. Julian McAuley from UCSD. We choose the smaller subset of the customer review data from the Kindle store of Amazon.com. The data is in the JSON format, which contains 982,619 reviews and metadata spanning May 1996 - July 2014.

In [5]:
# load original .json data
kindle_json = spark.read.json('/FileStore/tables/Kindle_Store_5.json')

In [6]:
kindle_json.show(3)

+----------+-------+-------+--------------------+----------+--------------+------------+------------------+--------------+
 asin|helpful|overall| reviewText|reviewTime| reviewerID|reviewerName| summary|unixReviewTime|
+----------+-------+-------+--------------------+----------+--------------+------------+------------------+--------------+
B000F83SZQ| [0, 0]| 5.0|I enjoy vintage b...|05 5, 2014|A1F6404F1VG29J| Avidreader|Nice vintage story| 1399248000|
B000F83SZQ| [2, 2]| 4.0|This book is a re...|01 6, 2014| AN0N05A9LIJEQ| critters| Different...| 1388966400|
B000F83SZQ| [2, 2]| 4.0|This was a fairly...|04 4, 2014| A795DMNCJILA6| dot| Oldie| 1396569600|
+----------+-------+-------+--------------------+----------+--------------+------------+------------------+--------------+
only showing top 3 rows

### Generate Sentiment Label

Reviews with overall rating of 1, 2, or 3 are labeled as negative (label=1), and reviews with overall rating of 4 or 5 are labeled as positive (label=0).

In [8]:
kindle_json.createOrReplaceTempView('kindle_json_view')

data_json = spark.sql('''
  SELECT CASE WHEN overall<4 THEN 1
          ELSE 0
          END as label,
        reviewText as text
  FROM kindle_json_view
  WHERE length(reviewText)>2''')

data_json.groupBy('label').count().show()

+-----+------+
label| count|
+-----+------+
 1|153331|
 0|829250|
+-----+------+

### Generate the dataset for modeling
We only sample a small portion of the data for demonstration and try to balance the two classes.

In [10]:
# Sampling data
pos = data_json.where('label=0').sample(False, 0.05, seed=1220)
neg = data_json.where('label=1').sample(False, 0.25, seed=1220)
data = pos.union(neg)
data.groupBy('label').count().show()

+-----+-----+
label|count|
+-----+-----+
 1|38542|
 0|41226|
+-----+-----+

In [11]:
# Negative reviews are on average longer than the positive reviews, but not significantly longer
from pyspark.sql.functions import length
data.withColumn('review_length', length('text')).groupBy('label').avg('review_length').show()

+-----+------------------+
label|avg(review_length)|
+-----+------------------+
 1| 622.5214311660008|
 0| 597.164071217193|
+-----+------------------+

### Data Preprocessing
Data preprocessing process uses the following steps:

* Use HTMLParser to un-escape the text
* Change "can't" to "can not", and change "n't" to "not" (This is useful for the negation handling process)
* Pad punctuations with blanks
* Lowercase every word
* Word tokenization
* Word lemmatization
* Perform **negation handling**
    * Use a state variable to store the negation state
    * Transform a word followed by a "not" or "no" into “not_” + word
    * Whenever the negation state variable is set, the words read are treated as “not_” + word
    * The state variable is reset when a punctuation mark is encountered or when there is double negation
* Use **bigram** and/or **trigram** models

In [13]:
# Define preprocessing function
def clean(text):
    import html
    import string
    import nltk
    nltk.download('wordnet')
    
    line = html.unescape(text)
    line = line.replace("can't", 'can not')
    line = line.replace("n't", " not")
    # Pad punctuations with white spaces
    pad_punct = str.maketrans({key: " {0} ".format(key) for key in string.punctuation}) 
    line = line.translate(pad_punct)
    line = line.lower()
    line = line.split() 
    lemmatizer = nltk.WordNetLemmatizer()
    line = [lemmatizer.lemmatize(t) for t in line] 
    
    # Negation handling
    # Add "not_" prefix to words behind "not", or "no" until the end of the sentence
    tokens = []
    negated = False
    for t in line:
        if t in ['not', 'no']:
            negated = not negated
        elif t in string.punctuation or not t.isalpha():
            negated = False
        else:
            tokens.append('not_' + t if negated else t)
    
    invalidChars = str(string.punctuation.replace("_", ""))  
    bi_tokens = list(nltk.bigrams(line))
    bi_tokens = list(map('_'.join, bi_tokens))
    bi_tokens = [i for i in bi_tokens if all(j not in invalidChars for j in i)]
    tri_tokens = list(nltk.trigrams(line))
    tri_tokens = list(map('_'.join, tri_tokens))
    tri_tokens = [i for i in tri_tokens if all(j not in invalidChars for j in i)]
    tokens = tokens + bi_tokens + tri_tokens      
    
    return tokens

In [14]:
# An example: how the function clean() pre-processes the input text
example = clean("I don't think this book has any decent information!!! It is full of typos and factual errors that I can't ignore.")
print(example)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Unzipping corpora/wordnet.zip.
['i', 'do', 'not_think', 'not_this', 'not_book', 'not_ha', 'not_any', 'not_decent', 'not_information', 'it', 'is', 'full', 'of', 'typo', 'and', 'factual', 'error', 'that', 'i', 'can', 'not_ignore', 'i_do', 'do_not', 'not_think', 'think_this', 'this_book', 'book_ha', 'ha_any', 'any_decent', 'decent_information', 'it_is', 'is_full', 'full_of', 'of_typo', 'typo_and', 'and_factual', 'factual_error', 'error_that', 'that_i', 'i_can', 'can_not', 'not_ignore', 'i_do_not', 'do_not_think', 'not_think_this', 'think_this_book', 'this_book_ha', 'book_ha_any', 'ha_any_decent', 'any_decent_information', 'it_is_full', 'is_full_of', 'full_of_typo', 'of_typo_and', 'typo_and_factual', 'and_factual_error', 'factual_error_that', 'error_that_i', 'that_i_can', 'i_can_not', 'can_not_ignore']

In [15]:
# Perform data preprocessing
from pyspark.sql.functions import udf, col, size
from pyspark.sql.types import ArrayType, StringType
clean_udf = udf(clean, ArrayType(StringType()))
data_tokens = data.withColumn('tokens', clean_udf(col('text')))
data_tokens.show(3)

+-----+--------------------+--------------------+
label| text| tokens|
+-----+--------------------+--------------------+
 0|I've been an occa...|[i, ve, been, an,...|
 0|There is so much ...|[there, is, so, m...|
 0|Love this! This ...|[love, this, this...|
+-----+--------------------+--------------------+
only showing top 3 rows

### Split dataset to training (70%) and testing (30%) sets

In [17]:
# Split data to 70% for training and 30% for testing
training, testing = data_tokens.randomSplit([0.7,0.3], seed=1220)
training.groupBy('label').count().show()

+-----+-----+
label|count|
+-----+-----+
 1|26978|
 0|28737|
+-----+-----+

In [18]:
training.cache()

Out[ 12 ]: DataFrame[label: int, text: string, tokens: array<string>]

### Naive Bayes Model (with parameter tuning)

In [20]:
from pyspark.ml.feature import CountVectorizer, IDF
from pyspark.ml import Pipeline

count_vec = CountVectorizer(inputCol='tokens', outputCol='c_vec', minDF=5.0)
idf = IDF(inputCol="c_vec", outputCol="features")

In [21]:
# Naive Bayes model
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes()

pipeline_nb = Pipeline(stages=[count_vec, idf, nb])

model_nb = pipeline_nb.fit(training)
test_nb = model_nb.transform(testing)
test_nb.show(3)

+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
label| text| tokens| c_vec| features| rawPrediction| probability|prediction|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
 0|"A Calm Whisper":...|[a, calm, whisper...|(237599,[0,1,2,3,...|(237599,[0,1,2,3,...|[-35891.606184157...| [1.0,0.0]| 0.0|
 0|"A Work in Progre...|[a, work, in, pro...|(237599,[0,1,2,3,...|(237599,[0,1,2,3,...|[-11666.318940416...|[1.0,2.1104994083...| 0.0|
 0|"It's impossible ...|[it, s, impossibl...|(237599,[0,1,2,3,...|(237599,[0,1,2,3,...|[-11361.684192180...|[1.0,2.1713068771...| 0.0|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
only showing top 3 rows

#### Naive Bayes model performance (using default parameters)
* Area under the ROC curve: 0.8551
* Accuracy: 0.8553

In [23]:
# Naive Bayes model ROC
from pyspark.ml.evaluation import BinaryClassificationEvaluator
roc_nb_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='label')
roc_nb = roc_nb_eval.evaluate(test_nb)
print("ROC of the NB model: {}".format(roc_nb))

ROC of the NB model: 0.8550587747934197

In [24]:
# Naive Bayes model accuracy
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
acc_nb_eval = MulticlassClassificationEvaluator(metricName='accuracy')
acc_nb = acc_nb_eval.evaluate(test_nb)
print("Accuracy of the NB model: {}".format(acc_nb))

Accuracy of the NB model: 0.8552779279092005

#### Naive Bayes model performance after parameter tuning
* CountVectorizer.minDF = 7.0
* NaiveBayes.smooting = 1.0
* Accuracy: 0.8568 (increased from 0.8553)

In [26]:
# NB parameter tuning and CV
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

paramGrid_nb = (ParamGridBuilder()
                .addGrid(count_vec.minDF, [3.0, 5.0, 7.0, 10.0, 15.0])
                .addGrid(nb.smoothing, [0.1, 0.5, 1.0])
                .build())
cv_nb = CrossValidator(estimator=pipeline_nb, estimatorParamMaps=paramGrid_nb, evaluator=acc_nb_eval, numFolds=5)
cv_model_nb = cv_nb.fit(training) 

In [27]:
test_cv_nb = cv_model_nb.transform(testing)
acc_nb_cv = acc_nb_eval.evaluate(test_cv_nb)
print("Accuracy of the NB CV model: {}".format(acc_nb_cv))

Accuracy of the NB CV model: 0.8568161975637134

In [28]:
cv_model_nb.bestModel.stages[0].extractParamMap()

Out[ 44 ]: 
{Param(parent='CountVectorizer_44cda3f9b64132bd062e', name='inputCol', doc='input column name'): 'tokens',
 Param(parent='CountVectorizer_44cda3f9b64132bd062e', name='binary', doc='If True, all non zero counts are set to 1.'): False,
 Param(parent='CountVectorizer_44cda3f9b64132bd062e', name='vocabSize', doc='max size of the vocabulary'): 262144,
 Param(parent='CountVectorizer_44cda3f9b64132bd062e', name='minDF', doc='Specifies the minimum number of different documents a term must appear in to be included in the vocabulary. If this is an integer >= 1, this specifies the number of documents the term must appear in; if this is a double in [0,1), then this specifies the fraction of documents.'): 7.0,
 Param(parent='CountVectorizer_44cda3f9b64132bd062e', name='outputCol', doc='output column name'): 'c_vec',
 Param(parent='CountVectorizer_44cda3f9b64132bd062e', name='minTF', doc="Filter to ignore rare words in a document. For each document, terms with frequency/count less than the given threshold are ignored. If this is an integer >= 1, then this specifies a count (of times the term must appear in the document); if this is a double in [0,1), then this specifies a fraction (out of the document's token count). Note that the parameter is only used in transform of CountVectorizerModel and does not affect fitting."): 1.0}

In [29]:
cv_model_nb.bestModel.stages[2].extractParamMap()

Out[ 45 ]: 
{Param(parent='NaiveBayes_43c0b7665b211d226a53', name='predictionCol', doc='prediction column name'): 'prediction',
 Param(parent='NaiveBayes_43c0b7665b211d226a53', name='rawPredictionCol', doc='raw prediction (a.k.a. confidence) column name'): 'rawPrediction',
 Param(parent='NaiveBayes_43c0b7665b211d226a53', name='probabilityCol', doc='Column name for predicted class conditional probabilities. Note: Not all models output well-calibrated probability estimates! These probabilities should be treated as confidences, not precise probabilities'): 'probability',
 Param(parent='NaiveBayes_43c0b7665b211d226a53', name='modelType', doc='The model type which is a string (case-sensitive). Supported options: multinomial (default) and bernoulli.'): 'multinomial',
 Param(parent='NaiveBayes_43c0b7665b211d226a53', name='smoothing', doc='The smoothing parameter.'): 1.0,
 Param(parent='NaiveBayes_43c0b7665b211d226a53', name='labelCol', doc='label column name'): 'label',
 Param(parent='NaiveBayes_43c0b7665b211d226a53', name='featuresCol', doc='features column name'): 'features'}

### Logistic Regressions
Model performance (using default parameters)
* Area under the ROC curve: 0.8601
* Accuracy: 0.8610

In [31]:
# Logistic Regression model
from pyspark.ml.classification import LogisticRegression
lgr = LogisticRegression(maxIter=5)
pipeline_lgr = Pipeline(stages=[count_vec, idf, lgr])

model_lgr = pipeline_lgr.fit(training)
test_lgr = model_lgr.transform(testing)

In [32]:
# Logistic Regression model ROC
from pyspark.ml.evaluation import BinaryClassificationEvaluator
roc_lgr_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='label')
roc_lgr = roc_lgr_eval.evaluate(test_lgr)
print("ROC of the model: {}".format(roc_lgr))

ROC of the model: 0.8601176818374295

In [33]:
# Logistic Regression model accuracy
#from pyspark.ml.evaluation import MulticlassClassificationEvaluator
acc_lgr_eval = MulticlassClassificationEvaluator(metricName='accuracy')
acc_lgr = acc_lgr_eval.evaluate(test_lgr)
print("Accuracy of the model: {}".format(acc_lgr))

Accuracy of the model: 0.8609736831164512

### Linear SVC Model
Model performance (using default parameters)
* Area under the ROC curve: 0.8649
* Accuracy: 0.8656

In [35]:
# Linear SVC model
from pyspark.ml.classification import LinearSVC
lsvc = LinearSVC(maxIter=5)
pipeline_lsvc = Pipeline(stages=[count_vec, idf, lsvc])

model_lsvc = pipeline_lsvc.fit(training)
test_lsvc = model_lsvc.transform(testing)

In [36]:
# Linear SVC model ROC
from pyspark.ml.evaluation import BinaryClassificationEvaluator
roc_lsvc_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='label')
roc_lsvc = roc_lsvc_eval.evaluate(test_lsvc)
print("ROC of the model: {}".format(roc_lsvc))

ROC of the model: 0.8648930463858352

In [37]:
# Linear SVC model accuracy
#from pyspark.ml.evaluation import MulticlassClassificationEvaluator
acc_lsvc_eval = MulticlassClassificationEvaluator(metricName='accuracy')
acc_lsvc = acc_lsvc_eval.evaluate(test_lsvc)
print("Accuracy of the model: {}".format(acc_lsvc))

Accuracy of the model: 0.8656300669355174

### Predict on new reviews:
To demonstrate the model prediction on new review texts, I randomly choose five reviews from the Kindle book *The Brave Ones: A Memoir of Hope, Pride and Military Service, by Michael J. MacLeod*. 

The suffixes "_1", "_2", ..., "_5" indicate the real overall review stars 1, 2, ..., 5.

The model correctly predicts the first three reviews as "negative" (label=1), and the last two as "positive" (label=0).

In [39]:
review_1 = ["WOW!!! No words describe how bland this book is. It took me a lot to even pick up to read. I would definitely not recommend this book."]

In [40]:
review_2 = ["A first person account of the war in Afghanistan. It skipps around a lot and is like a never-ending news article. On the positive side, you do get a feel for what desert fighting is like from a soldiers point of view."]

In [41]:
review_3 = ["I liked the premise and most of the book. At the end parts I lost a little interest because I lost the thread of who was who. War is hell. MacLeod did his service unlike most of us."]

In [42]:
review_4 = ["Very informative first person account of the the daily life of a US Paratrooper. From training to deployment in combat situations in Afghanistan. Well worth the read and makes you really understand and appreciate their sacrifices"]

In [43]:
review_5 = ["This is perhaps the best wrote book I have ever read. Articulate and thought provoking. Not just a riveting account of actual combat, but Michael was able to do what few before him have...captured the essence of what one feels as the battle unfolds. Perhaps most of all, I am grateful to call this author 'Fellow Warrior' Airborne all the way!!!"]

In [44]:
from pyspark.sql.types import *
schema = StructType([StructField("text", StringType(), True)])

text = [review_1, review_2, review_3, review_4, review_5]
review_new = spark.createDataFrame(text, schema=schema)

In [45]:
# Data preprocessing
review_new_tokens = review_new.withColumn('tokens', clean_udf(col('text')))
review_new_tokens.show()

+--------------------+--------------------+
 text| tokens|
+--------------------+--------------------+
WOW!!! No words d...|[wow, not_word, n...|
A first person ac...|[a, first, person...|
I liked the premi...|[i, liked, the, p...|
Very informative ...|[very, informativ...|
This is perhaps t...|[this, is, perhap...|
+--------------------+--------------------+

In [46]:
# Prediction using tuned Naive Bayes model
result = cv_model_nb.transform(review_new_tokens)
result.select('text', 'prediction').show()

+--------------------+----------+
 text|prediction|
+--------------------+----------+
WOW!!! No words d...| 1.0|
A first person ac...| 1.0|
I liked the premi...| 1.0|
Very informative ...| 0.0|
This is perhaps t...| 0.0|
+--------------------+----------+